# 1. softmax回归的简洁实现

通过深度学习框架的高级API能够使实现softmax回归变得更容易

In [1]:
import torch 
from torch import nn
import d2l_torch as d2l

batch_size=256
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size)

softmax回归的 输出层 是一个全连接层

In [ ]:
# pytorch不会隐式的帮助我们调整输入的形状
# 因此，定义展平层(flatten)在线性层前调整网络输入的形状

net=nn.Sequential(nn.Flatten(),nn.Linear(784,10))



In [ ]:
nn.Conv2d(1, 32, 5, 1, 1)